In [15]:
%matplotlib widget

In [16]:
import matplotlib.pyplot as plt
import numpy as np
from math import pi, sin, cos, acos, tan, atan, radians 

In [17]:
def define_circle(p1, p2, p3):
    """
    Returns the center and radius of the circle passing the given 3 points.
    In case the 3 points form a line, returns (None, infinity).
    """
    temp = p2[0] * p2[0] + p2[1] * p2[1]
    bc = (p1[0] * p1[0] + p1[1] * p1[1] - temp) / 2
    cd = (temp - p3[0] * p3[0] - p3[1] * p3[1]) / 2
    det = (p1[0] - p2[0]) * (p2[1] - p3[1]) - (p2[0] - p3[0]) * (p1[1] - p2[1])

    if abs(det) < 1.0e-6:
        return (None, np.inf)

    # Center of circle
    cx = (bc*(p2[1] - p3[1]) - cd*(p1[1] - p2[1])) / det
    cy = ((p1[0] - p2[0]) * cd - (p2[0] - p3[0]) * bc) / det

    radius = np.sqrt((cx - p1[0])**2 + (cy - p1[1])**2)
    return ((cx, cy), radius)

In [18]:
m = 1.0            # module
z = 60             # number of teeth
a0 = radians(20.0) # pressure angle
c = 0.0           # clearance

d0 = m * z         # pitch diameter
da = m * (z + 2.0) # addendum circle diameter
dd = m * (z - 2.0) - 2.0 * c # dedendum circle diameter
s0 = m * (pi / 2.0 + 0.0 * tan(a0)) # tooth thickness on the pitch circle
inv_a0 = tan(a0) - a0

r0 = d0 / 2.0
ra = da / 2.0
rd = dd / 2.0

npoints = 300

t = np.linspace(0.0, pi * 2.0, npoints)

db = np.cos(a0) * d0
d = np.linspace(db, da, npoints)

cos_a = d0 / d * np.cos(a0)
# cos_a[np.where(cos_a > 1.0)] = 1.0
a = np.arccos(cos_a)
inv_a = np.tan(a) - a
s = d * (s0 / d0 + inv_a0 - inv_a)
phi = (s / 2.0) / (d / 2.0)

def gear_points():
    curve_points = 20
    
    rb = np.cos(a0) * d0 / 2.0 # base circle radius
    r = np.linspace(rb, ra, curve_points)
    cos_a = r0 / r * np.cos(a0)
    a = np.arccos(cos_a)
    inv_a = np.tan(a) - a
    s = r * (s0 / d0 + inv_a0 - inv_a)
    phi = s / r
    
    x = np.cos(phi) * r
    y = np.sin(phi) * r
    
    b = np.linspace(phi[-1], -phi[-1], curve_points)
    x = np.concatenate((x, np.cos(b) * ra))
    y = np.concatenate((y, np.sin(b) * ra))
    
    x = np.concatenate((x, (np.cos(-phi) * r)[::-1]))
    y = np.concatenate((y, (np.sin(-phi) * r)[::-1]))
    
    tau = np.pi * 2.0 / z
    rho = tau - phi[0] * 2.0
    
    p1 = (x[-1], y[-1])
    p2 = (np.cos(-phi[0] - rho / 2.0) * rd,
          np.sin(-phi[0] - rho / 2.0) * rd)
    p3 = (np.cos(-phi[0] - rho) * rb,
          np.sin(-phi[0] - rho) * rb)
    
    bcxy, bcr = define_circle(p1, p2, p3)
        
    t1 = np.arctan2(p1[1] - bcxy[1], p1[0] - bcxy[0])
    t2 = np.arctan2(p3[1] - bcxy[1], p3[0] - bcxy[0])
    
    if t1 < 0.0:
        t1 += np.pi * 2.0
    if t2 < 0.0:
        t2 += np.pi * 2.0
        
    t1, t2 = min(t1, t2), max(t1, t2)
    
    t = np.linspace(t1 + np.pi * 2.0, t2 + np.pi * 2.0, curve_points)
    
    x = np.concatenate((x, bcxy[0] + bcr * np.cos(t)))
    y = np.concatenate((y, bcxy[1] + bcr * np.sin(t)))
    
    res_x, res_y = x.copy(), y.copy()
    
    angle = -tau
    for i in range(z - 1):
        res_x = np.concatenate((res_x, x * np.cos(angle) - y * np.sin(angle)))
        res_y = np.concatenate((res_y, x * np.sin(angle) + y * np.cos(angle)))
        angle -= tau
        
    return res_x, res_y

fig = plt.figure()
plt.plot(np.cos(t) * r0, np.sin(t) * r0)
plt.plot(np.cos(t) * ra, np.sin(t) * ra)
plt.plot(np.cos(t) * rd, np.sin(t) * rd)

gx, gy = gear_points()

plt.plot(gx, gy)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
fig = plt.figure()
plt.plot((np.cos(t) + t * np.sin(t)),  (np.sin(t) - t * np.cos(t)))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
def senv(theta0, theta):
    res = 1.0 / np.sin(theta0) * np.arccos(np.cos(theta) / np.cos(theta0))
    res -= np.arccos(np.tan(theta0) / np.tan(theta))
    return res

def sphere_to_cartesian(r, theta, phi):
    return (r * np.sin(theta) * np.cos(phi),
            r * np.sin(theta) * np.sin(phi),
            r * -np.cos(theta))

module = 1.0
teeth_number = 16
tooth_width = 5.0
partial_cone_angle = np.radians(45.0)
pressure_angle = np.radians(20.0)
helix_angle = np.radians(30.0)

d_outside = module * teeth_number;                # Part Cone Diameter at the Cone Base,
                                                  # corresponds to the Chord in a Spherical Section
r_outside = d_outside / 2.0
rg_outside = r_outside / np.sin(partial_cone_angle) # Large-Cone Radius for Outside-Tooth, corresponds to the Length of the Cone-Flank;
rg_inside = rg_outside - tooth_width                # Large-Cone Radius for Inside-Tooth  
r_inside = r_outside * rg_inside/rg_outside
c = module / 6.0                            # Tip Clearance
df_outside = d_outside - (module + c) * 2.0 * np.cos(partial_cone_angle)
rf_outside = df_outside / 2.0

if module < 1.0:
    da_outside = d_outside + module * 2.2 * np.cos(partial_cone_angle)
else:
    da_outside = d_outside + module * 2 * np.cos(partial_cone_angle)

ra_outside = da_outside / 2.0
delta_a = np.arcsin(ra_outside / rg_outside)

alpha_spur = np.arctan(np.tan(pressure_angle) / np.cos(helix_angle)) # Helix Angle in Transverse Section
delta_b = np.arcsin(np.cos(alpha_spur) * np.sin(partial_cone_angle)) # Base Cone Angle  
delta_f = np.arcsin(rf_outside / rg_outside)

phi_r = senv(delta_b, partial_cone_angle) # Angle to Point of Involute on Partial Cone
mirrpoint = np.pi / teeth_number + 2.0 * phi_r

height_f = rg_outside * np.cos(delta_f)

start = max(delta_b, delta_f)

step = (delta_a - delta_b) / 16.0;

x, y, z = np.ndarray(0), np.ndarray(0), np.ndarray(0)
px, py, pz = np.ndarray(0), np.ndarray(0), np.ndarray(0)

for tau in np.linspace(0.0, np.pi * 2.0, teeth_number):
    delta = np.arange(start, delta_a, step)
    phis = senv(delta_b, delta)
    rs = np.full(delta.shape[0], rg_outside)
    
    x1, y1, z1 = sphere_to_cartesian(rs, delta, phis + tau)
    x2, y2, z2 = sphere_to_cartesian(rs, delta, mirrpoint - phis + tau)
    
    x = np.concatenate((x, x1, x2[::-1]))
    y = np.concatenate((y, y1, y2[::-1]))
    z = np.concatenate((z, z1 + height_f, z2[::-1] + height_f))
    
    rp = height_f / np.cos(delta)
    
    px1, py1, pz1 = sphere_to_cartesian(rp, delta, phis + tau)
    px2, py2, pz2 = sphere_to_cartesian(rp, delta, mirrpoint - phis + tau)
    
    px = np.concatenate((px, px1, px2[::-1]))
    py = np.concatenate((py, py1, py2[::-1]))
    pz = np.concatenate((pz, pz1 + height_f, pz2[::-1] + height_f))

In [14]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.set_xlim3d(-10.0, 10.0)
ax.set_ylim3d(-10.0, 10.0)
ax.set_zlim3d(0.0, 10.0)
ax.plot3D(x, y, z)
ax.plot3D(px, py, pz)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
t = np.linspace(0.0, np.pi * 2, 400)

x = np.cos(t) * 10.0
y = np.sin(t) * 10.0
z = np.sin(t * 16.0) * 0.1

fig = plt.figure()
ax = plt.axes(projection='3d')

ax.plot3D(x, y, z)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [109]:
print(delta_f, delta_b)

0.6869605922069126 0.7100548958779229
